In [2]:
import pandas as pd
import numpy as np
import datetime as dt

In [3]:
import pickle

In [215]:
df_raw = pd.read_csv('metadata.csv')

In [216]:
df_raw.head(3)

,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,Microsoft Academic Paper ID,WHO #Covidence,has_full_text,full_text_file
0,NaN,Elsevier,Intrauterine virus infections and congenital h...,10.1016/0002-8703(72)90077-4,NaN,4361535.0,els-covid,Abstract The etiologic basis for the vast majo...,1972-12-31,"Overall, James C.",American Heart Journal,NaN,NaN,False,custom_license
1,NaN,Elsevier,Coronaviruses in Balkan nephritis,10.1016/0002-8703(80)90355-5,NaN,6243850.0,els-covid,NaN,1980-03-31,"Georgescu, Leonida; Diosi, Peter; Buţiu, Ioan;...",American Heart Journal,NaN,NaN,False,custom_license
2,NaN,Elsevier,Cigarette smoking and coronary heart disease: ...,10.1016/0002-8703(80)90356-7,NaN,7355701.0,els-covid,NaN,1980-03-31,"Friedman, Gary D",American Heart Journal,NaN,NaN,False,custom_license


In [217]:
df_raw.shape

(44220, 15)

In [218]:
len(df_raw.loc[:, 'abstract'].dropna())

35806

In [219]:
has_abstract = ~df_raw.loc[:, 'abstract'].isna()

In [239]:
has_title = ~df_raw.loc[:, 'title'].isna()

In [240]:
columns = ['title', 'authors', 'abstract', 'publish_time', 'journal']
papers = df_raw[has_abstract & has_title].reset_index(drop=True).loc[:, columns]

In [241]:
papers

,title,authors,abstract,publish_time,journal
0,Intrauterine virus infections and congenital h...,"Overall, James C.",Abstract The etiologic basis for the vast majo...,1972-12-31,American Heart Journal
1,Clinical and immunologic studies in identical ...,"Brunner, Carolyn M.; Horwitz, David A.; Shann,...","Abstract Middle-aged female identical twins, o...",1973-08-31,The American Journal of Medicine
2,Epidemiology of community-acquired respiratory...,"Garibaldi, Richard A.",Abstract Upper respiratory tract infections ar...,1985-06-28,The American Journal of Medicine
3,Infectious diarrhea: Pathogenesis and risk fac...,"Cantey, J.Robert",Abstract Our understanding of the pathogenesis...,1985-06-28,The American Journal of Medicine
4,New perspectives on the pathogenesis of rheuma...,"Zvaifler, Nathan J.",Abstract In the pathogenesis of rheumatoid art...,1988-10-14,The American Journal of Medicine
...,...,...,...,...,...
35652,Epidemiology and clinical profile of pathogens...,"Brini, Ines; Guerrero, Aida; Hannachi, Naila; ...",This study aimed to identify a broad spectrum ...,2017 Nov 17,PLoS One
35653,Functional Analysis of the Transmembrane Domai...,"Bissonnette, Mei Lin Z.; Donald, Jason E.; DeG...","Abstract To enter cells, enveloped viruses use...",2009-02-13,Journal of Molecular Biology
35654,Viral Entry Properties Required for Fitness in...,"Iketani, Sho; Shean, Ryan C.; Ferren, Marion; ...",Human parainfluenza viruses cause a large burd...,2018 Jul 3,mBio
35655,Arenavirus reverse genetics: New approaches fo...,"Emonet, Sebastien E.; Urata, Shuzo; de la Torr...","Abstract Several arenaviruses, chiefly Lassa v...",2011-03-15,Virology


In [9]:
pd.to_datetime(papers.loc[:, 'publish_time'])

ParserError: Unknown string format: 2006 Jun-Dec

In [15]:
papers.iloc[3]['abstract']

'Abstract Our understanding of the pathogenesis of infectious, especially bacterial, diarrhea has increased dramatically. New etiologic agents, mechanisms, and diseases have become known. For example, Escherichia coli serogroup 0157 is now known to cause acute hemorrhagic colitis. Also, E. coli serogroups that produce Shiga toxin are recognized as etiologic agents in the hemolytic-uremic syndrome. The production of bacterial diarrhea has two major facets, bacterial-mucosal interaction and the induction of intestinal fluid loss by enterotoxins. Bacterial-mucosal interaction can be described in stages: (1) adherence to epithelial cell microvilli, which is often promoted by or associated with pill; (2) close adherence (enteroadherence), usually by classic enteropathogenic E. coli, to mucosal epithelial cells lacking microvilli; and (3) mucosal invasion, as with Shigella and Salmonella infections. Further large strides in understanding infectious diarrhea are likely with the cloning of vir

In [20]:
papers.iloc[3]['title']

'Infectious diarrhea: Pathogenesis and risk factors'

In [21]:
df_2 = papers.head(100)

In [26]:
(df_2['abstract'] + 3 * (' '+ df_2['title']))[0]

'Abstract The etiologic basis for the vast majority of cases of congenital heart disease remains largely undefined. Viruses have been considered to be likely candidates since the recognition of the association between intrauterine rubella and congenital heart disease. Although the pathogenesis of cardiovascular defects is poorly understood, information gained from the study of congenital rubella syndrome suggests that mechanisms such as focal endothelial cell damage, resulting in obliteration of vascular supply, decreased growth rate, and shortened survival time of certain cells, and disturbed DNA replication in cells whose chromosomes were damaged secondary to the effects of virus replication may be operative in the production of defects in the developing fetus. In addition to rubella there is suggestive, but not conclusive, evidence that Coxsackie B3 and B4 virus infections during pregnancy can result in the birth of infants with a variety of types of congenital heart lesions and tha

In [321]:
papers.to_parquet('cord_papers.parquet')

## Corona API

In [28]:
import requests

In [402]:
response = requests.get('https://corona-api.com/countries')

In [403]:
country_data = response.json()['data']

In [404]:
len(country_data)

249

In [405]:
AF = country_data[0]

In [406]:
AF

{'coordinates': {'latitude': 33, 'longitude': 65},
 'name': 'Afghanistan',
 'code': 'AF',
 'population': 29121286,
 'updated_at': '2020-04-02T13:48:54.152Z',
 'today': {'deaths': 0, 'confirmed': 2},
 'latest_data': {'deaths': 4,
  'confirmed': 239,
  'recovered': 10,
  'critical': 0,
  'calculated': {'death_rate': 1.6736401673640167,
   'recovery_rate': 4.184100418410042,
   'recovered_vs_death_ratio': None,
   'cases_per_million_population': 6}}}

Parse the information on a country into a useful format

In [407]:
trial_dict = dict()

In [408]:
code = AF.pop('code')

In [409]:
latest_data = AF.pop('latest_data')
latest_data

{'deaths': 4,
 'confirmed': 239,
 'recovered': 10,
 'critical': 0,
 'calculated': {'death_rate': 1.6736401673640167,
  'recovery_rate': 4.184100418410042,
  'recovered_vs_death_ratio': None,
  'cases_per_million_population': 6}}

In [410]:
calculated = latest_data.pop('calculated')
calculated

{'death_rate': 1.6736401673640167,
 'recovery_rate': 4.184100418410042,
 'recovered_vs_death_ratio': None,
 'cases_per_million_population': 6}

In [411]:
tmp = AF.pop('today')
today = {
    'deaths_today': tmp['deaths'],
    'confirmed_today': tmp['confirmed']
}

In [412]:
AF.pop('coordinates')

{'latitude': 33, 'longitude': 65}

In [413]:
AF

{'name': 'Afghanistan',
 'population': 29121286,
 'updated_at': '2020-04-02T13:48:54.152Z'}

In [414]:
AF.update(latest_data)

In [415]:
AF.update(calculated)

In [416]:
AF.update(today)

In [417]:
AF

{'name': 'Afghanistan',
 'population': 29121286,
 'updated_at': '2020-04-02T13:48:54.152Z',
 'deaths': 4,
 'confirmed': 239,
 'recovered': 10,
 'critical': 0,
 'death_rate': 1.6736401673640167,
 'recovery_rate': 4.184100418410042,
 'recovered_vs_death_ratio': None,
 'cases_per_million_population': 6,
 'deaths_today': 0,
 'confirmed_today': 2}

In [418]:
trial_dict[code] = AF

In [419]:
trial_dict

{'AF': {'name': 'Afghanistan',
  'population': 29121286,
  'updated_at': '2020-04-02T13:48:54.152Z',
  'deaths': 4,
  'confirmed': 239,
  'recovered': 10,
  'critical': 0,
  'death_rate': 1.6736401673640167,
  'recovery_rate': 4.184100418410042,
  'recovered_vs_death_ratio': None,
  'cases_per_million_population': 6,
  'deaths_today': 0,
  'confirmed_today': 2}}

### Turn into a function

In [420]:
import typing

In [421]:
def process_covid_data(data: typing.List[dict]) -> dict:
    master = dict()
    
    for country in data:
        code = country.pop('code')
        info = country.pop('latest_data')
        rates = info.pop('calculated')
        
        # Need to change the keys in today's data due to overlap with info
        tmp = country.pop('today')
        today = {'deaths_today': tmp['deaths'], 'confirmed_today': tmp['confirmed']}
        
        # We don't care about the coordinates
        country.pop('coordinates')
        
        country.update(info)
        country.update(rates)
        country.update(today)
        
        master[code] = country
    
    return master

In [422]:
data_dict = process_covid_data(response.json()['data'])

In [423]:
pd.DataFrame(data_dict)

,AF,AL,AX,AS,DZ,AD,AO,AI,AG,AQ,...,SE,TZ,TO,TC,GB,VU,WF,ZM,EH,YE
name,Afghanistan,Albania,Åland Islands,American Samoa,Algeria,Andorra,Angola,Anguilla,Antigua and Barbuda,Antarctica,...,Sweden,"Tanzania, United Republic of",Tonga,Turks and Caicos Islands,UK,Vanuatu,Wallis and Futuna,Zambia,Western Sahara,Yemen
population,29121286,2986952,26711,57881,34586184,84000,13068161,13254,86754,None,...,9828655,41892895,122580,20556,62348447,221552,16025,13460305,273008,23495361
updated_at,2020-04-02T13:48:54.152Z,2020-04-02T13:48:54.152Z,2020-04-02T13:48:54.152Z,2020-04-02T13:48:54.152Z,2020-04-02T13:48:54.152Z,2020-04-02T13:48:54.152Z,2020-04-02T13:48:54.152Z,2020-04-02T13:48:54.152Z,2020-04-02T13:48:54.152Z,2020-04-02T13:48:54.152Z,...,2020-04-02T13:48:54.152Z,2020-04-02T13:48:54.152Z,2020-04-02T13:48:54.152Z,2020-04-02T13:48:54.152Z,2020-04-02T13:48:54.152Z,2020-04-02T13:48:54.152Z,2020-04-02T13:48:54.152Z,2020-04-02T13:48:54.152Z,2020-04-02T13:48:54.152Z,2020-04-02T13:48:54.152Z
deaths,4,16,0,0,58,15,2,0,0,0,...,282,0,0,0,2921,0,0,1,0,0
confirmed,239,277,0,0,847,428,8,2,7,0,...,5466,0,0,0,33718,0,0,39,0,0
recovered,10,67,0,0,61,10,1,0,0,0,...,103,0,0,0,135,0,0,0,0,0
critical,0,7,0,0,0,12,0,0,0,0,...,429,0,0,0,163,0,0,0,0,0
death_rate,1.67364,5.77617,None,None,6.8477,3.50467,25,0,0,None,...,5.15917,None,None,None,8.66303,None,None,2.5641,None,None
recovery_rate,4.1841,24.1877,None,None,7.20189,2.33645,12.5,0,0,None,...,1.88438,None,None,None,0.40038,None,None,0,None,None
recovered_vs_death_ratio,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [424]:
covid_df = pd.DataFrame(data_dict).T.rename(columns={'name': 'country'})

In [425]:
covid_df.loc['CA']

country                                           Canada
population                                      33679000
updated_at                      2020-04-02T13:48:54.152Z
deaths                                               129
confirmed                                           9731
recovered                                           1736
critical                                             120
death_rate                                       1.32566
recovery_rate                                    17.8399
recovered_vs_death_ratio                            None
cases_per_million_population                         258
deaths_today                                          15
confirmed_today                                        0
Name: CA, dtype: object

In [399]:
pd.to_datetime(covid_df.loc[:, 'updated_at'])

AF   2020-03-28 15:10:07.640000+00:00
AL   2020-03-28 15:10:07.640000+00:00
AX   2020-03-28 15:10:07.640000+00:00
AS   2020-03-28 15:10:07.640000+00:00
DZ   2020-03-28 15:10:07.640000+00:00
                   ...               
VU   2020-03-28 15:10:07.640000+00:00
WF   2020-03-28 15:10:07.640000+00:00
ZM   2020-03-28 15:10:07.640000+00:00
EH   2020-03-28 15:10:07.640000+00:00
YE   2020-03-28 15:10:07.640000+00:00
Name: updated_at, Length: 249, dtype: datetime64[ns, UTC]

In [401]:
covid_df.head(-1)

,country,population,updated_at,deaths,confirmed,recovered,critical,death_rate,recovery_rate,recovered_vs_death_ratio,cases_per_million_population,deaths_today,confirmed_today
AF,Afghanistan,29121286,2020-03-28T15:10:07.640Z,4,110,2,0,None,None,None,3,0,0
AL,Albania,2986952,2020-03-28T15:10:07.640Z,10,197,31,3,None,None,None,68,2,11
AX,Åland Islands,26711,2020-03-28T15:10:07.640Z,0,0,0,0,None,None,None,0,0,0
AS,American Samoa,57881,2020-03-28T15:10:07.640Z,0,0,0,0,None,None,None,0,0,0
DZ,Algeria,34586184,2020-03-28T15:10:07.640Z,26,409,29,0,None,None,None,9,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
GB,UK,62348447,2020-03-28T15:10:07.640Z,1019,17089,135,163,None,None,None,252,260,2546
VU,Vanuatu,221552,2020-03-28T15:10:07.640Z,0,0,0,0,None,None,None,0,0,0
WF,Wallis and Futuna,16025,2020-03-28T15:10:07.640Z,0,0,0,0,None,None,None,0,0,0
ZM,Zambia,13460305,2020-03-28T15:10:07.640Z,0,28,0,0,None,None,None,2,0,6


In [426]:
covid_df.loc['US']

country                                              USA
population                                     310232863
updated_at                      2020-04-02T13:48:54.152Z
deaths                                              5113
confirmed                                         215362
recovered                                           8878
critical                                            5005
death_rate                                       2.37414
recovery_rate                                    4.12236
recovered_vs_death_ratio                            None
cases_per_million_population                         651
deaths_today                                          11
confirmed_today                                      359
Name: US, dtype: object

In [367]:
covid_df.sort_values(by='confirmed', ascending=False)

,name,population,updated_at,deaths,confirmed,recovered,critical,death_rate,recovery_rate,recovered_vs_death_ratio,cases_per_million_population,deaths_today,confirmed_today
US,USA,310232863,2020-03-28T15:10:07.640Z,1717,105019,2537,2494,None,None,None,317,21,893
IT,Italy,60340328,2020-03-28T15:10:07.640Z,9134,86498,10950,3732,None,None,None,1431,0,0
CN,China,1330044000,2020-03-28T15:10:07.640Z,3295,81394,74971,886,None,None,None,57,3,54
ES,Spain,46505963,2020-03-28T15:10:07.640Z,5690,72248,12285,4165,None,None,None,1545,552,6529
DE,Germany,81802257,2020-03-28T15:10:07.640Z,399,53340,6658,1581,None,None,None,637,48,2469
...,...,...,...,...,...,...,...,...,...,...,...,...,...
SB,Solomon Islands,559198,2020-03-28T15:10:07.640Z,0,0,0,0,None,None,None,0,0,0
SL,Sierra Leone,5245695,2020-03-28T15:10:07.640Z,0,0,0,0,None,None,None,0,0,0
ST,Sao Tome and Principe,175808,2020-03-28T15:10:07.640Z,0,0,0,0,None,None,None,0,0,0
TW,"Taiwan, Province of China",22894384,2020-03-28T15:10:07.640Z,0,0,0,0,None,None,None,0,0,0


In [381]:
covid_df.loc[:, 'name']

AF          Afghanistan
AL              Albania
AX        Åland Islands
AS       American Samoa
DZ              Algeria
            ...        
VU              Vanuatu
WF    Wallis and Futuna
ZM               Zambia
EH       Western Sahara
YE                Yemen
Name: name, Length: 249, dtype: object

In [396]:
for row in covid_df.loc[:, ['name']].sort_values('name').iterrows():
    print(row[0])
    print(row[1]['name'])
#     print(row[1]['name'])
#     print(row[1]['population'])

AF
Afghanistan
AL
Albania
DZ
Algeria
AS
American Samoa
AD
Andorra
AO
Angola
AI
Anguilla
AQ
Antarctica
AG
Antigua and Barbuda
AR
Argentina
AM
Armenia
AW
Aruba
AU
Australia
AT
Austria
AZ
Azerbaijan
BS
Bahamas
BH
Bahrain
BD
Bangladesh
BB
Barbados
BY
Belarus
BE
Belgium
BZ
Belize
BJ
Benin
BM
Bermuda
BT
Bhutan
BO
Bolivia, Plurinational State of
BQ
Bonaire, Sint Eustatius and Saba
BA
Bosnia and Herzegovina
BW
Botswana
BV
Bouvet Island
BR
Brazil
IO
British Indian Ocean Territory
BN
Brunei Darussalam
BG
Bulgaria
BF
Burkina Faso
BI
Burundi
KH
Cambodia
CM
Cameroon
CA
Canada
CV
Cape Verde
KY
Cayman Islands
CF
Central African Republic
TD
Chad
CL
Chile
CN
China
CX
Christmas Island
CC
Cocos (Keeling) Islands
CO
Colombia
KM
Comoros
CG
Congo
CD
Congo, the Democratic Republic of the
CK
Cook Islands
CR
Costa Rica
HR
Croatia
CU
Cuba
CW
Curaçao
CY
Cyprus
CZ
Czechia
CI
Côte d'Ivoire
DK
Denmark
DJ
Djibouti
DM
Dominica
DO
Dominican Republic
EC
Ecuador
EG
Egypt
SV
El Salvador
GQ
Equatorial Guinea
ER
Eritrea
EE

In [4]:
sorted(zip(covid_df.index, covid_df.loc[:, 'name']))[:20]

NameError: name 'covid_df' is not defined

# Modeling

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import MiniBatchKMeans

In [211]:
model = MiniBatchKMeans(800, batch_size=1000, verbose=3)

In [212]:
tfidf_vectorizer = TfidfVectorizer(stop_words='english')

In [231]:
papers.loc[:, 'title'].isna().sum()

149

In [232]:
papers.loc[:, 'abstract'].isna().sum()

0

In [237]:
pd.isna(np.nan)

True

In [254]:
vectorize_text = (papers.loc[:, 'title'] + ' ') * 5 + papers.loc[:, 'abstract']

In [255]:
tfidf = tfidf_vectorizer.fit_transform(vectorize_text)

In [256]:
model.fit(tfidf)

Init 1/3 with method: k-means++
Inertia for init 1/3: 1965.153104
Init 2/3 with method: k-means++
Inertia for init 2/3: 1960.290659
Init 3/3 with method: k-means++
Inertia for init 3/3: 1997.332757
Minibatch iteration 1/3600: mean batch inertia: 0.932802, ewa inertia: 0.932802 
Minibatch iteration 2/3600: mean batch inertia: 0.885884, ewa inertia: 0.930171 
Minibatch iteration 3/3600: mean batch inertia: 0.875921, ewa inertia: 0.927128 
Minibatch iteration 4/3600: mean batch inertia: 0.891407, ewa inertia: 0.925124 
Minibatch iteration 5/3600: mean batch inertia: 0.877301, ewa inertia: 0.922442 
Minibatch iteration 6/3600: mean batch inertia: 0.877780, ewa inertia: 0.919937 
Minibatch iteration 7/3600: mean batch inertia: 0.874723, ewa inertia: 0.917401 
Minibatch iteration 8/3600: mean batch inertia: 0.862947, ewa inertia: 0.914347 
Minibatch iteration 9/3600: mean batch inertia: 0.858994, ewa inertia: 0.911242 
[MiniBatchKMeans] Reassigning 500 cluster centers.
Minibatch iteration 10

MiniBatchKMeans(batch_size=1000, compute_labels=True, init='k-means++',
                init_size=None, max_iter=100, max_no_improvement=10,
                n_clusters=800, n_init=3, random_state=None,
                reassignment_ratio=0.01, tol=0.0, verbose=3)

In [250]:
sample_abs = df.loc[:10, 'abstract']

In [251]:
sample_titles = df.loc[:10, 'title']

In [257]:
model.predict(tfidf_vectorizer.transform(sample_titles))

Computing label assignment and total inertia


array([467, 467, 179,  18, 467, 179, 317, 467, 467, 178, 487], dtype=int32)

In [258]:
model.predict(tfidf_vectorizer.transform(sample_abs))

Computing label assignment and total inertia


array([467, 467, 319, 514, 467, 179,  94, 102, 467, 467, 487], dtype=int32)

## Look at cosine similarity

In [276]:
from sklearn.metrics.pairwise import cosine_similarity

In [260]:
print(tfidf_vectorizer.transform(sample_titles))

  (0, 99717)	0.1445550377974342
  (0, 51314)	0.6485587168025844
  (0, 49888)	0.2197133571266129
  (0, 44443)	0.4454823426791145
  (0, 30874)	0.18144841025312333
  (0, 24659)	0.5280373371123137
  (1, 96588)	0.4782600107836734
  (1, 91986)	0.24621369639459478
  (1, 90322)	0.14667752871650472
  (1, 56977)	0.3899279367561479
  (1, 48825)	0.3479271605327876
  (1, 47879)	0.27428657939354956
  (1, 35560)	0.41133181744225406
  (1, 30806)	0.3899279367561479
  (1, 22959)	0.13987132631296137
  (2, 95074)	0.29992396848198094
  (2, 81534)	0.18316175007798247
  (2, 49888)	0.21006871953285142
  (2, 49382)	0.34204266930588617
  (2, 49013)	0.2892471028063903
  (2, 36043)	0.39591765341488855
  (2, 35140)	0.3369507132763349
  (2, 23982)	0.32327775702842093
  (2, 9588)	0.35707790219979
  (2, 8947)	0.35973842929650984
  :	:
  (8, 44443)	0.2226216822067147
  (8, 42479)	0.4147207986872343
  (8, 36604)	0.21485396627650905
  (8, 27516)	0.28357518186708863
  (8, 25055)	0.16082304336622597
  (8, 17959)	0.1855915

In [261]:
model.labels_

array([467, 467, 319, ..., 208, 390,  32], dtype=int32)

Find cosine similarity between elements in cluster 467

In [269]:
title_467 = papers.loc[0, 'title']
title_467

'Intrauterine virus infections and congenital heart disease'

In [270]:
abs_467 = papers.loc[0, 'abstract']
abs_467

'Abstract The etiologic basis for the vast majority of cases of congenital heart disease remains largely undefined. Viruses have been considered to be likely candidates since the recognition of the association between intrauterine rubella and congenital heart disease. Although the pathogenesis of cardiovascular defects is poorly understood, information gained from the study of congenital rubella syndrome suggests that mechanisms such as focal endothelial cell damage, resulting in obliteration of vascular supply, decreased growth rate, and shortened survival time of certain cells, and disturbed DNA replication in cells whose chromosomes were damaged secondary to the effects of virus replication may be operative in the production of defects in the developing fetus. In addition to rubella there is suggestive, but not conclusive, evidence that Coxsackie B3 and B4 virus infections during pregnancy can result in the birth of infants with a variety of types of congenital heart lesions and tha

In [283]:
vector_sample_467 = tfidf_vectorizer.transform([title_467 * 3 + abs_467])

In [359]:
model.predict(vector_sample_467)

Computing label assignment and total inertia


array([467], dtype=int32)

In [338]:
cluster_467 = model.labels_[1:] == 467

#### try with multiple clusters

In [339]:
cluster_390 = model.labels_[1:] == 390

In [342]:
cluster_467 | cluster_390

array([ True, False, False, ..., False,  True, False])

In [346]:
li = [390, 467]
li_mask = np.zeros(len(model.labels_ - 1), dtype=bool)
for c in li:
    li_mask = li_mask | (model.labels_ == c)
li_mask

array([ True,  True, False, ..., False,  True, False])

In [280]:
vector_467 = tfidf_vectorizer.transform(vectorize_text[1:][cluster_467])

In [1]:
print(vector_467)

NameError: name 'vector_467' is not defined

In [296]:
cosine_similarity(vector_467, vector_sample_467)

array([[0.00481547],
       [0.02680058],
       [0.00078486],
       ...,
       [0.00318941],
       [0.001941  ],
       [0.01888669]])

In [298]:
df_clust_467 = papers[1:][cluster_467]

In [299]:
df_clust_467['sim'] = cosine_similarity(vector_467, vector_sample_467)

In [303]:
df_clust_467.sort_values('sim', ascending=False)

,title,authors,abstract,publish_time,journal,sim
15516,Congenital Malaria in China,"Tao, Zhi-yong; Fang, Qiang; Liu, Xue; Culleton...","ABSTRACT: BACKGROUND: Congenital malaria, in w...",2014 Mar 13,PLoS Negl Trop Dis,0.316365
24439,Harnessing fetal and adult genetic reprogramin...,"Nandi, Shyam Sundar; Mishra, Paras Kumar",Heart is the first organ formed during organog...,NaN,NaN,0.304875
9074,Trends in the utilization of computed tomograp...,"Yang, Justin Cheng-Ta; Lin, Ming-Tai; Jaw, Fu-...",Background/Purpose Pediatric cardiac computed ...,2015-11-30,Journal of the Formosan Medical Association,0.288445
29372,Neonatal congenital microvillus atrophy,"Pecache, N; Patole, S; Hagan, R; Hill, D; Char...",Congenital microvillous atrophy (CMVA) is the ...,NaN,NaN,0.263464
23191,Airway ciliary dysfunction: Association with a...,"Stewart, Eileen; Adams, Phillip S.; Tian, Xin;...",OBJECTIVE: Heterotaxy (HTX) congenital heart d...,NaN,NaN,0.248664
...,...,...,...,...,...,...
3865,First synthesis of 4-(arylsulfonyl)phenols by ...,"Riahi, Abdolmajid; Shkoor, Mohanad; Fatunsin, ...","Abstract The formal [3+3] cyclization of 1,3-b...",2009-01-07,Tetrahedron Letters,0.000000
12137,Pandemic: tracking contagions Sonia Shah Sarah...,"Burki, Talha Khan",Unknown,2016-05-31,The Lancet Respiratory Medicine,0.000000
3864,Synthesis of a library of glycosylated flavonols,"Li, Zhitao; Ngojeh, George; DeWitt, Paul; Zhen...",Abstract Flavonols are an important class of n...,2008-12-15,Tetrahedron Letters,0.000000
28496,Ten years of fighting bird flu,"Parry, Jane",This year marks a decade since the first human...,NaN,NaN,0.000000


### Convert to function

In [351]:
def find_similar_articles(title: typing.List[str], abstract: typing.List[str], 
                          cluster_label: typing.List[int]) -> pd.DataFrame:
    
    this_cluster = model.labels_ == cluster_label
    similar_papers = papers.loc[this_cluster, ['title', 'abstract', 'authors', 'journal']]
    this_vector = tfidf_vectorizer.transform([title * 3 + abstract])
    that_vector = tfidf_vectorizer.transform(similar_papers.loc[:, 'title'] * 3 
                                              + similar_papers.loc[:, 'abstract'])
    similarities = cosine_similarity(that_vector, this_vector)
    similar_papers['sim'] = similarities
    
    similar_papers = similar_papers.sort_values('sim', ascending=False)
    
    # slice from [1:] since the first value will be the same value that was entered
    if np.isclose(similar_papers.loc[0, 'sim'], 1):
        return similar_papers.iloc[1:].head(50)  
    else: 
        return similar_papers.head(50)

In [353]:
recommendations = pd.DataFrame()

In [355]:
pd.concat([recommendations, find_similar_articles(title_467, abs_467, 467)], ignore_index=True)

,title,abstract,authors,journal,sim
0,Harnessing fetal and adult genetic reprogramin...,Heart is the first organ formed during organog...,"Nandi, Shyam Sundar; Mishra, Paras Kumar",NaN,0.309782
1,Congenital Malaria in China,"ABSTRACT: BACKGROUND: Congenital malaria, in w...","Tao, Zhi-yong; Fang, Qiang; Liu, Xue; Culleton...",PLoS Negl Trop Dis,0.263407
2,Trends in the utilization of computed tomograp...,Background/Purpose Pediatric cardiac computed ...,"Yang, Justin Cheng-Ta; Lin, Ming-Tai; Jaw, Fu-...",Journal of the Formosan Medical Association,0.253924
3,Neonatal congenital microvillus atrophy,Congenital microvillous atrophy (CMVA) is the ...,"Pecache, N; Patole, S; Hagan, R; Hill, D; Char...",NaN,0.210476
4,Airway ciliary dysfunction: Association with a...,OBJECTIVE: Heterotaxy (HTX) congenital heart d...,"Stewart, Eileen; Adams, Phillip S.; Tian, Xin;...",NaN,0.204812
5,Neuro-invasion by a ‘Trojan Horse’ strategy an...,The central nervous system (CNS) is a major ta...,"Bielefeldt-Ohmann, Helle; Smirnova, Natalia P;...",NaN,0.185748
6,Molecular Biology of Rubella Virus,Publisher Summary This chapter summarizes the ...,"Frey, Teryl K.",Advances in Virus Research,0.175224
7,The utility of margin-reflex distance in deter...,AIMS: To evaluate the utility of margin-reflex...,"Ural, Ozlem; Mocan, Mehmet Cem; Dolgun, Anıl; ...",Indian J Ophthalmol,0.170991
8,Birth Defects Data from Surveillance Hospitals...,BACKGROUND: To determine the prevalence and ch...,"Tu, Lh; Li, H; Zhang, Hp; Li, Xd; Lin, Jj; Xio...",Iran J Public Health,0.170037
9,Congenital Cytomegalovirus Infection Alters Ol...,"In developed countries, cytomegalovirus (CMV)-...","Lazarini, Françoise; Katsimpardi, Lida; Levivi...",NaN,0.164100


In [248]:
with open ('covid_model.pkl', 'wb') as f:
    pickle.dump(model, f)

In [249]:
with open('covid_tfidf_vectorizer.pkl', 'wb') as f:
    pickle.dump(tfidf_vectorizer, f)